# Document Agent Prototype

This notebook contains examples for using the Document Agent Prototype.

It uses the new doc_agent_chat and doc_agent_upload services to upload documents to a project and then asks an agent questions about the documents.

In [8]:
import json
import subprocess
import os

## Upload a document

We can upload a document to the database by calling the doc_agent_upload service.

This service takes a URL to fetch text content, then it processes the text and uploads it to a vector database (Pinecone).

In [9]:
# The new services are designed to be called in a similar way to the existing AI chat services. This is just a function to illustrate the usage in this notebook.
def upload_document(doc_url, user_description, project_id):
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
    services_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
    
    input_data = {
        "doc_url": doc_url,
        "user_description": user_description,
        "project_id": project_id
    }
    
    input_path = os.path.join(services_dir, "tmp", "doc_agent_upload.json")
    output_path = os.path.join(services_dir, "tmp", "upload_output.json")
    
    with open(input_path, "w") as f:
        json.dump(input_data, f, indent=2)
    
    result = subprocess.run(
        ["python", "entry.py", "doc_agent_upload", "--input", "tmp/doc_agent_upload.json", "--output", "tmp/upload_output.json"],
        capture_output=True,
        text=True,
        cwd=services_dir
    )
    
    with open(output_path, "r") as f:
        output_data = json.load(f)
    
    print(f"✓ Uploaded: {user_description}")
    return output_data

The service takes a URL, user description and project ID. It returns a document ID for the uploaded document.

In [ ]:
# Example: upload a document
doc_url = "https://raw.githubusercontent.com/OpenFn/docs/refs/heads/main/docs/design/design-workflow.md"
user_description = "Design workflow"
project_id = "proj_1"

# Uncomment to upload:
# upload_result = upload_document(doc_url, user_description, project_id)

✓ Uploaded: Design workflow


## Chat with the document agent

We can chat to an LLM that has the ability to query the uploaded documents by calling the doc_agent_chat service.

In [10]:
def call_agent_chat(content, project_id, project_name, documents, history):
    # Build the input payload
    input_data = {
        "content": content,
        "context": {
            "project_id": project_id,
            "project_name": project_name,
            "documents": documents
        },
        "history": history
    }
    
    # Get the services directory (one level up from doc_agent_chat)
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
    services_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
    
    input_path = os.path.join(services_dir, "tmp", "doc_agent_chat.json")
    output_path = os.path.join(services_dir, "tmp", "output.json")
    
    # Write input file
    with open(input_path, "w") as f:
        json.dump(input_data, f, indent=2)
    
    # Call the service from the services directory
    result = subprocess.run(
        ["python", "entry.py", "doc_agent_chat", "--input", "tmp/doc_agent_chat.json", "--output", "tmp/output.json"],
        capture_output=True,
        text=True,
        cwd=services_dir
    )
    
    # Read the output
    with open(output_path, "r") as f:
        output_data = json.load(f)
    
    # Print only the response text
    if "response" in output_data:
        print("Response:")
        print("-" * 80)
        print(output_data["response"])
        print("-" * 80)
    
    return output_data

The document agent service takes a user message (content), project ID, project name, documents and conversation history as inputs.

Like the other Apollo chat services, this service is stateless and should be called every conversation turn. The front-end would need to have a system for storing project and document IDs at least, and the database a more sophisticated way to limit access between orgs/projects/docs.

In [11]:
# Query parameters
content = "How should I handle user data when working with the CLI?"
project_id = "proj_1"
project_name = "OpenFN CLI Project"

# All uploaded documents that the user wants to include in the conversation
documents = [
    {
        "uuid": "fd8370c4-24ad-4323-8ab2-4053569aedef",
        "title": "CLI Walkthrough",
        "description": "OpenFn CLI documentation"
    }
]
# Or use empty list if no documents:
# documents = []

history = []

# Call the service and save output
output = call_agent_chat(content, project_id, project_name, documents, history)

Response:
--------------------------------------------------------------------------------
Based on the documentation, here are the key practices for handling user data when working with the CLI:

## Security Best Practices

### 1. **Store Sensitive Files in a `tmp` Directory**



It's a good practice to store `state.json` and `output.json` in a `tmp` folder

, as these files often contain sensitive information.

### 2. **Never Upload Sensitive Files to GitHub**



Since `state.json` and `output.json` may contain sensitive configuration information and project data, it's important to never upload them to GitHub. To ensure that GitHub ignores these files, add the `tmp` directory to your `.gitignore` file

 using: `echo "tmp" >> .gitignore`

### 3. **Be Careful with Logging**



Note that `console.log(state)` will display the whole state, including `state.configuration` elements such as **username and password**. Remove this log whenever you're done debugging to avoid accidentally exposi

## Inspect Output Details

Explore tool calls, retrieved texts, and other details:

In [7]:
output

{'code': 500,
 'type': 'INTERNAL_ERROR',
 'message': "string indices must be integers, not 'str'"}

In [ ]:
# Tool calls
if "tool_calls" in output:
    print("Tool Calls:")
    print("-" * 80)
    for i, tool_call in enumerate(output["tool_calls"], 1):
        print(f"\n{i}. {tool_call.get('name', 'Unknown')}")
        if "arguments" in tool_call:
            print(f"   Arguments: {json.dumps(tool_call['arguments'], indent=2)}")
    print("-" * 80)

# Retrieved texts/documents
if "retrieved_texts" in output:
    print("\n\nRetrieved Texts:")
    print("-" * 80)
    for i, text in enumerate(output["retrieved_texts"], 1):
        print(f"\n{i}. {text[:200]}..." if len(text) > 200 else f"\n{i}. {text}")
    print("-" * 80)

# Show all available keys
print("\n\nAvailable output keys:", list(output.keys()))